In [1]:
from src.util_tools import compute_cross_entropy, compute_ortho_loss
from tools.project import INPUT_PATH, LOGS_PATH, OUTPUT_PATH, MODELS_PATH

import audiocraft
from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
import torch
from torch.onnx.symbolic_opset9 import tensor
from torchviz import make_dot
import typing as tp
from audiocraft.modules.conditioners import ConditioningAttributes
import tqdm
import torch
from audiocraft.data.audio import audio_read, audio_write
from audiocraft.data.audio_utils import convert_audio_channels, convert_audio
import umap
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from torch.utils.tensorboard import SummaryWriter
from sklearn.decomposition import PCA
from random import shuffle
from torch.utils.data import TensorDataset, random_split, DataLoader
from audioldm_eval.metrics.fad import FrechetAudioDistance
import os
import contextlib
import io
import torchaudio
def count_directories(path):
    import os
    if not os.path.exists(path):
        return 0
    return sum(os.path.isdir(os.path.join(path, entry)) for entry in os.listdir(path))

letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
EXP_NUM = count_directories(LOGS_PATH('textual-inversion'))+1
EXAMPLES_LEN = 5
BATCH_SIZE = 5
N_TOKENS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [2]:
model = MusicGen.get_pretrained('facebook/musicgen-small')
encodec = model.compression_model

/home/ubuntu/.cache/pypoetry/virtualenvs/musicgen-WI8jtfXt-py3.10/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
tokens = torch.load(INPUT_PATH("textual-inversion-v3", "data", "train","8bit", "encoded", "music_p15.pt")).to(DEVICE)
tokens = tokens.squeeze()
n, k = tokens.shape
padding = torch.ones((n, 400 - k), device=tokens.device, dtype=torch.int64)*model.lm.special_token_id
tokens = torch.cat((tokens, padding), dim=1).detach()[None]
tokens

tensor([[[1062,  996,  988,  ..., 2048, 2048, 2048],
         [1182,  616, 1834,  ..., 2048, 2048, 2048],
         [1125, 1900, 1372,  ..., 2048, 2048, 2048],
         [ 753,   76,  850,  ..., 2048, 2048, 2048]]], device='cuda:0')

In [4]:
tokens = torch.load(INPUT_PATH("textual-inversion-v3", "data", "train","8bit", "encoded", "music_p15.pt")).to(DEVICE)
tokens = tokens.squeeze()
n, k = tokens.shape
padding = torch.ones((n, 400 - k), device=tokens.device, dtype=torch.int64)*0
tokens = torch.cat((tokens, padding), dim=1).detach()[None]
display_audio(encodec.decode(tokens, None),model.cfg.sample_rate )

In [8]:
set([torch.load(INPUT_PATH("textual-inversion-v3", "data", "train","8bit", "encoded", name)).shape[2] for name in
os.listdir(INPUT_PATH("textual-inversion-v3", "data", "train","8bit", "encoded"))])

{36, 250, 251}